Load user ratings from the CSV file.

In [ ]:
import pandas as pd
ratings_df = pd.read_csv('ratings.csv')

Load anime details from the CSV file.

In [ ]:
anime_df = pd.read_csv('anime.csv')

Merge user ratings and anime details into a single DataFrame.

In [ ]:
df = pd.merge(ratings_df, anime_df, on='anime_id')

Create a subset DataFrame with selected anime IDs.

In [ ]:
sample_df = df[df['anime_id'].isin(selected_anime_ids)]

Create a user-item pivot table.

In [ ]:
user_anime_df = sample_df.pivot(index='user_id', columns='anime_id', values='rating')

Convert the DataFrame to Surprise Dataset format.

In [ ]:
from surprise import Dataset
from surprise import Reader
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(sample_df[['user_id', 'anime_id', 'rating']], reader)

Split the data into training and test datasets.

In [ ]:
from surprise.model_selection import train_test_split
t_trainset, testset = train_test_split(data, test_size=0.2)

Set up GridSearchCV for hyperparameter tuning.

In [ ]:
from surprise import SVD
from surprise.model_selection import GridSearchCV
param_grid = {'n_factors': [50, 100], 'n_epochs': [5, 10]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

Fit the model using the training dataset.

In [ ]:
grid_search.fit(t_trainset)

Retrieve the best SVD model from the grid search.

In [ ]:
svd_model = grid_search.best_estimator_

Generate predicted ratings for the test dataset.

In [ ]:
predictions = svd_model.predict(testset)